In [ ]:
!pip install python-dotenv
!pip install azure-search-documents

from dotenv import load_dotenv
from openai import AzureOpenAI
import os

# Load variables from .env file
load_dotenv()

# Access the variables
openai_endpoint = os.environ.get('OPENAI_ENDPOINT')
openai_key = os.environ.get('OPENAI_KEY')
aisearch_endpoint = os.environ.get('AISEARCH_ENDPOINT')
aisearch_key = os.environ.get('AISEARCH_KEY')

openai_client = AzureOpenAI(
  api_key = openai_key,
  api_version = "2024-02-15-preview",
  azure_endpoint = openai_endpoint
)

In [8]:
# Create an assistant
assistant = openai_client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4" #You must replace this value with the deployment name for your model.
)

print(assistant.model_dump_json(indent=2))

# Create a thread
thread = openai_client.beta.threads.create()
print(thread)

# Add a user question to the thread
message = openai_client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)

# Add a user question to the thread
message = openai_client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)

thread_messages = openai_client.beta.threads.messages.list(thread.id)

run = openai_client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)


{
  "id": "asst_egFHjDzQhxjcErLOadHRFWbf",
  "created_at": 1713738921,
  "description": null,
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "gpt-4",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ],
  "response_format": null,
  "temperature": null,
  "tool_resources": null,
  "top_p": null,
  "file_ids": []
}
Thread(id='thread_NuwyYvroMz4J4lROxizm8qnD', created_at=171

In [14]:
import time
from IPython.display import clear_output
import json
from PIL import Image

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = openai_client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = openai_client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))

data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)  # Outputs: assistant-1YGVTvNzc2JXajI5JU9F0HMD
content = openai_client.files.content(image_file_id)

image= content.write_to_file("sinewave.png")
# Display the image in the default image viewer
image = Image.open("sinewave.png")
image.show()


Status: completed
Elapsed time: 0 minutes 0 seconds
{
  "data": [
    {
      "id": "msg_i82T3jG3Z7UVgmQmmk6UVw4I",
      "assistant_id": "asst_egFHjDzQhxjcErLOadHRFWbf",
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-qimhDHLrbp2Ij2a2dRzInlhS"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the visualization of a sine wave. The x-axis represents the angle in radians, and the y-axis represents the sine of that angle."
          },
          "type": "text"
        }
      ],
      "created_at": 1713738934,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_QWIVUQnRpIjHKVY12ft3pmvt",
      "status": null,
      "thread_id": "thread_NuwyYvroMz4J4lROxizm8qnD",
      "file_ids": []
    

In [15]:
image = Image.open("sinewave.png")
image.show()

In [19]:
# Add a new user question to the thread
message = openai_client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to generate the sinewave"
)
run = openai_client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)

# Retrieve the status of the run
run = openai_client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

messages = openai_client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))



queued
{
  "data": [
    {
      "id": "msg_YDWTko9sZrmUUiGsPFn22mcP",
      "assistant_id": null,
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Show me the code you used to generate the sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 1713739798,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_NuwyYvroMz4J4lROxizm8qnD",
      "file_ids": []
    },
    {
      "id": "msg_0x6PwOYj8OveYKC3MoOUbYKX",
      "assistant_id": "asst_egFHjDzQhxjcErLOadHRFWbf",
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Certainly, here is the code I used to create the sine wave visualization:\

In [21]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

Show me the code you used to generate the sinewave


: 